# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Цель исследования:**
Построение модели для задачи классификации, которая выберет подходящий тариф «Смарт» или «Ультра» для клиентов мобильной связи «Мегалайн».

**Ход исследования:**
1. Загрузка данных
2. Подготовка данных
3. Разделение данных на выборки: Исходные данные были разделены на обучающую (70%), валидационную (15%) и тестовую (15%) выборки.
4. Исследование качества разных моделей:Логистическая регрессия, Решающее дерево и Случайный лес
5. Выбор лучшей модели

Проверка качества модели на тестовой выборке: Выбранная модель Случайного леса была проверена на тестовой выборке, и была рассчитана точность (accuracy) на тестовой выборке.

**Описание данных** `users_behavior.csv`
* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

## Откройте и изучите файл

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [4]:
df.describe()

,calls,minutes,messages,mb_used,is_ultra
count,3214.000000,3214.000000,3214.000000,3214.000000,3214.000000
mean,63.038892,438.208787,38.281269,17207.673836,0.306472
std,33.236368,234.569872,36.148326,7570.968246,0.461100
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,40.000000,274.575000,9.000000,12491.902500,0.000000
50%,62.000000,430.600000,30.000000,16943.235000,0.000000
75%,82.000000,571.927500,57.000000,21424.700000,1.000000
max,244.000000,1632.060000,224.000000,49745.730000,1.000000


In [5]:
#df.isna().sum().sort_values(ascending=False) #проверка пропусков 
#df.hist(figsize=(15, 20))

**Вывод**
- Общий объем данных 3214 записей
- Все столбцы имеют соответствующий тип данных
- Пропущенные значения отсутствуют
- Аномальные значения не выявлены

## Разбейте данные на выборки

параметр is_ultra является целевым признаком

In [6]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']

In [7]:
X_train, X_temp, y_train, y_temp = train_test_split(features, target, test_size=0.3, random_state=12345, stratify=target)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=12345, stratify=y_temp)

In [8]:
print("Размер обучающей выборки:", X_train.shape)
print("Размер валидационной выборки:", X_valid.shape)
print("Размер тестовой выборки:", X_test.shape)
print("Размер обучающей выборки:", y_train.shape)
print("Размер валидационной выборки:", y_valid.shape)
print("Размер тестовой выборки:", y_test.shape)

Размер обучающей выборки: (2249, 4)
Размер валидационной выборки: (482, 4)
Размер тестовой выборки: (483, 4)
Размер обучающей выборки: (2249,)
Размер валидационной выборки: (482,)
Размер тестовой выборки: (483,)


Исходные данные разделены на обучающую, валидационную и тестовую выборки. Параметр is_ultra является целевым признаком, который необходимо предсказать.

- Обучающая выборка (70% данных): X_train, y_train
- Валидационная выборка (15% данных): X_valid, y_valid
- Тестовая выборка (15% данных): X_test, y_test
- Полученные выборки готовы для обучения и оценки качества моделей

## Исследуйте модели

In [9]:
param_grid = {
    'C': [0.1, 1.0, 10.0],
    'class_weight': [None, 'balanced']
}

logistic_model = LogisticRegression(random_state=12345)
grid_search = GridSearchCV(logistic_model, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_accuracy = grid_search.best_score_

best_log_model = LogisticRegression(random_state=12345, **best_params)
best_log_model.fit(X_train, y_train)

best_log_accuracy = accuracy_score(y_valid, best_log_model.predict(X_valid))

print('Лучшие параметры для логистической регрессии:', best_params)
print('Точность на валидационной выборке:', best_log_accuracy)

Лучшие параметры для логистической регрессии: {'C': 0.1, 'class_weight': None}
Точность на валидационной выборке: 0.7053941908713693


In [10]:
logistic_hyperparameters = [
    {'solver': 'lbfgs', 'max_iter': 100, 'C': 0.1, 'class_weight': None},
    {'solver': 'liblinear', 'max_iter': 200, 'C': 0.1, 'class_weight': None},
    {'solver': 'saga', 'max_iter': 1000, 'C': 0.1, 'class_weight': None}   
]

best_logistic_model = None
best_logistic_accuracy = 0

for params in logistic_hyperparameters:
    model = LogisticRegression(random_state=12345, solver='saga', max_iter=10000)
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, predictions)
    if accuracy > best_logistic_accuracy:
        best_logistic_model = model
        best_logistic_accuracy = accuracy

print('Лучшая точность логистической регрессии:', best_logistic_accuracy)

Лучшая точность логистической регрессии: 0.7012448132780082


In [11]:
import itertools
import numpy as np

class_weight = [None, 'balanced']
c_list = np.linspace(1e-2, 12)

logistic_hyperparameters = [{'C': number, 'class_weight': weight} 
                                for weight, number in list(itertools.product(class_weight, c_list))]

for params in logistic_hyperparameters:
    model = LogisticRegression(random_state=12345, **params)
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, predictions)
    if accuracy > best_logistic_accuracy:
        best_logistic_model = model
        best_logistic_accuracy = accuracy

print('Лучшая точность логистической регрессии:', best_logistic_accuracy)
print('Лучшие параметры:', best_logistic_model.get_params())

Лучшая точность логистической регрессии: 0.7053941908713693
Лучшие параметры: {'C': 0.01, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 12345, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [12]:
# Дерево решений
decision_tree_hyperparameters = [3, 5, 7]

best_decision_tree_model = None
best_decision_tree_accuracy = 0

for depth in decision_tree_hyperparameters:
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, predictions)
    if accuracy > best_decision_tree_accuracy:
        best_decision_tree_model = model
        best_decision_tree_accuracy = accuracy

print('Лучшая точность дерева решений:', best_decision_tree_accuracy)


Лучшая точность дерева решений: 0.8070539419087137


In [13]:
random_forest_hyperparameters = [
    {'n_estimators': 10, 'max_depth': 5},
    {'n_estimators': 50, 'max_depth': 10},
    {'n_estimators': 100, 'max_depth': 15}
]

best_random_forest_model = None
best_random_forest_accuracy = 0

for params in random_forest_hyperparameters:
    model = RandomForestClassifier(random_state=12345, **params)
    model.fit(X_train, y_train)
    predictions = model.predict(X_valid)
    accuracy = accuracy_score(y_valid, predictions)
    if accuracy > best_random_forest_accuracy:
        best_random_forest_model = model
        best_random_forest_accuracy = accuracy

print('Лучшая точность случайного леса:', best_random_forest_accuracy)

Лучшая точность случайного леса: 0.8195020746887967


best_model = None
best_accuracy = 0
best_depth = None
best_est = None

models = {
    "Logistic Regression": best_logistic_accuracy,
    "Decision Tree": best_decision_tree_accuracy,
    "Random Forest": best_random_forest_accuracy
}

for model, accuracy in models.items():
    if accuracy > best_accuracy:
        best_model = model
        best_accuracy = accuracy

        if model == "Random Forest":
            best_depth = best_random_forest_model.get_params()['max_depth']
            best_est = best_random_forest_model.get_params()['n_estimators']

print("Лучшая модель:", best_model)
print("Точность (accuracy на валидационной выборке:", best_accuracy)
print("Лучшая глубина дерева:", best_depth)
print("Лучшее количество деревьев:", best_est)

In [14]:
best_model = None
best_accuracy = 0
best_depth = None
best_est = None

models = {
    "Logistic Regression": best_logistic_accuracy,
    "Decision Tree": best_decision_tree_accuracy,
    "Random Forest": best_random_forest_accuracy
}

for model, accuracy in models.items():
    if accuracy > best_accuracy:
        best_model = model
        best_accuracy = accuracy

        if model == "Random Forest":
            best_depth = best_random_forest_model.get_params()['max_depth']
            best_est = best_random_forest_model.get_params()['n_estimators']
        else:
            best_depth = None
            best_est = None

print("Лучшая модель:", best_model)
print("Точность (accuracy) на валидационной выборке:", best_accuracy)

if best_model == "Random Forest":
    print("Лучшая глубина дерева:", best_depth)
    print("Лучшее количество деревьев:", best_est)

Лучшая модель: Random Forest
Точность (accuracy) на валидационной выборке: 0.8195020746887967
Лучшая глубина дерева: 5
Лучшее количество деревьев: 10


**Выводы:**
На данном этапе исследовались три модели: логистическую регрессию, решающее дерево и случайный лес.
Лучшей моделью оказалась "Случайный лес" с точностью около 81% на валидационной выборке.

## Проверьте модель на тестовой выборке

In [15]:
best_random_forest_model = RandomForestClassifier(random_state=12345, n_estimators=10, max_depth=5)
best_random_forest_model.fit(X_train, y_train)
test_predictions = best_random_forest_model.predict(X_test)
test_accuracy = accuracy_score(y_test, test_predictions)

print("Точность (accuracy) на тестовой выборке:", test_accuracy)

Точность (accuracy) на тестовой выборке: 0.8136645962732919


## (бонус) Проверьте модели на адекватность

In [16]:
from sklearn.dummy import DummyClassifier

# Создаем объект и обучаем
dummy_model = DummyClassifier(strategy="most_frequent", random_state=12345)
dummy_model.fit(X_train, y_train)

# Прогнозы для валид и тест
dummy_valid_predict = dummy_model.predict(X_valid)
dummy_test_predict = dummy_model.predict(X_test)

# Считаем точность
dummy_valid_accuracy = accuracy_score(y_valid, dummy_valid_predict)
dummy_test_accuracy = accuracy_score(y_test, dummy_test_predict)

print("Точность на валидационной выборке:", dummy_valid_accuracy)
print("Точность на тестовой выборке:", dummy_test_accuracy)

Точность на валидационной выборке: 0.6929460580912863
Точность на тестовой выборке: 0.6935817805383023
